In [129]:
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import dask.dataframe as dd
from dask_ml.preprocessing import OneHotEncoder, DummyEncoder, Categorizer, LabelEncoder, StandardScaler

In [84]:
approved_data = dd.read_csv('../data/approved1.csv')

In [85]:
approved_data.dtypes

id                        int64
addr_state               object
annual_inc              float64
application_type         object
disbursement_method      object
dti                     float64
earliest_cr_line         object
emp_length               object
emp_title                object
fico_range_high         float64
fico_range_low          float64
grade                    object
home_ownership           object
initial_list_status      object
installment             float64
int_rate                float64
issue_d                  object
loan_amnt               float64
open_acc                float64
pub_rec                 float64
pub_rec_bankruptcies    float64
purpose                  object
sub_grade                object
term                     object
verification_status      object
zip_code                 object
loan_status              object
dtype: object

In [86]:
approved_data = approved_data.dropna()

In [87]:
len(approved_data)

1219475

In [88]:
y = approved_data.loan_status
X = approved_data[['addr_state', 'annual_inc', 'application_type', 'disbursement_method', 'dti', 'earliest_cr_line', 'emp_length', 'emp_title', 'fico_range_high', 'fico_range_low', 'grade',
'home_ownership', 'initial_list_status', 'installment', 'int_rate', 'issue_d', 'loan_amnt', 'open_acc', 'pub_rec', 'pub_rec_bankruptcies', 'purpose', 'sub_grade', 'term', 'verification_status']]

In [89]:
X.earliest_cr_line = X.earliest_cr_line.map_partitions(pd.to_datetime, meta=('earliest_cr_line', 'datetime64[ns]'))

In [90]:
X['mnths_since_first_credit'] = (pd.Timestamp.now() - X.earliest_cr_line).dt.days

In [91]:
X.mnths_since_first_credit.compute()

1        6564
2        7476
3        6960
4        8541
5        7598
         ... 
52600    5527
52601    6776
52602    9486
52603    8084
52604    6320
Name: mnths_since_first_credit, Length: 1219475, dtype: int64

In [93]:
X = X.drop(['earliest_cr_line', 'emp_title'], axis=1)

In [101]:
X.emp_length = X.emp_length.str.strip('<+ years').astype(int)

In [106]:
X = X.drop('issue_d', axis=1)

In [111]:
X.term = X.term.str.strip(' months')

In [112]:
X.verification_status.compute()

1           Not Verified
2           Not Verified
3           Not Verified
4        Source Verified
5               Verified
              ...       
52600       Not Verified
52601    Source Verified
52602       Not Verified
52603       Not Verified
52604       Not Verified
Name: verification_status, Length: 1219475, dtype: object

In [118]:
ce = Categorizer(columns=['addr_state', 'application_type', 'disbursement_method', 'emp_length', 'grade', 'home_ownership', 'initial_list_status', 'purpose', 'term', 'verification_status'])
X = ce.fit_transform(X)

In [126]:
de = DummyEncoder(columns=['addr_state', 'application_type', 'disbursement_method', 'emp_length', 'grade', 'home_ownership', 'initial_list_status', 'purpose', 'term', 'verification_status'])
X = de.fit_transform(X)

In [131]:
X = X.fillna(0)

In [135]:
le = LabelEncoder()
y = le.fit_transform(y)
y.compute()

array([1, 1, 0, ..., 1, 1, 1])

In [138]:
X.values.compute()

array([[30784.0, 16.96, 669.0, ..., 1, 0, 0],
       [65000.0, 14.29, 709.0, ..., 1, 0, 0],
       [65000.0, 10.84, 709.0, ..., 1, 0, 0],
       ...,
       [57000.0, 22.18, 739.0, ..., 1, 0, 0],
       [100000.0, 18.92, 754.0, ..., 1, 0, 0],
       [38500.0, 4.64, 679.0, ..., 1, 0, 0]], dtype=object)

In [139]:
ss = StandardScaler()
ss.fit_transform(X.values, y)

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [127]:
LDA = LinearDiscriminantAnalysis()



annual_inc                             float64
dti                                    float64
fico_range_high                        float64
fico_range_low                         float64
installment                            float64
                                        ...   
term_36                                  uint8
term_60                                  uint8
verification_status_Not Verified         uint8
verification_status_Source Verified      uint8
verification_status_Verified             uint8
Length: 109, dtype: object